In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import partial

In [62]:
#https://www.kaggle.com/datasets/snehangsude/audible-dataset

In [63]:
wd = os.getcwd()+"/"
audible_uncleaned = pd.read_csv(wd+"audible_uncleaned.csv")

In [64]:
audible_uncleaned.head()

,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [65]:
#name. Number of books can be extracted depending on the case
#author: remove written by and separate names
#narrator: idem
#time:convert to datetime64
#releasedate: convert to datetime 64
#language: --
#stars: separate stars and ratings
#price: convert to euros.

In [66]:
audible_uncleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         87489 non-null  object
 1   author       87489 non-null  object
 2   narrator     87489 non-null  object
 3   time         87489 non-null  object
 4   releasedate  87489 non-null  object
 5   language     87489 non-null  object
 6   stars        87489 non-null  object
 7   price        87489 non-null  object
dtypes: object(8)
memory usage: 5.3+ MB


In [67]:
audible_cleaned =pd.DataFrame()

In [68]:
audible_cleaned["name"]=audible_uncleaned["name"].copy()

In [69]:
def extract_text_left_of (strReference, serToProcess):
    return serToProcess.str.extract(r""+strReference+"(.*)")

In [70]:
def splitCamelCase(str):
 
    new_string="*"
    for i in str:
        if(i.isupper() and new_string[-1].isalpha()):
            new_string+="*"+i
        else:
            new_string+=i
    
    x= new_string.split("*")
    if '' in x:
       x.remove('')
    return " ".join(x)


In [71]:
audible_cleaned["author"]=audible_uncleaned["author"]\
                            .pipe((extract_text_left_of,"serToProcess"),strReference="Writtenby:")\
                            .pipe(np.vectorize(splitCamelCase))

In [72]:
""" def splitCamelCase(str):
    def addAsteriskInCaseChange(str):
        new_string="*"
        for i in str:
            if(i.isupper() and new_string[-1].isalpha()):
                new_string+="*"+i
            else:
                new_string+=i
        return new_string

    def splitStrOnAsterisk(str):
        return str.split("*")
    def removeEmpty(lst):
    #if '' in lst:
    #    x.remove('')
        return [n for n in str if not(n=='')]
    def joinListWithSpaces(lst):
        return " ".join(lst)
    return joinListWithSpaces(removeEmpty(splitStrOnAsterisk( addAsteriskInCaseChange(str) ))) """

' def splitCamelCase(str):\n    def addAsteriskInCaseChange(str):\n        new_string="*"\n        for i in str:\n            if(i.isupper() and new_string[-1].isalpha()):\n                new_string+="*"+i\n            else:\n                new_string+=i\n        return new_string\n\n    def splitStrOnAsterisk(str):\n        return str.split("*")\n    def removeEmpty(lst):\n    #if \'\' in lst:\n    #    x.remove(\'\')\n        return [n for n in str if not(n==\'\')]\n    def joinListWithSpaces(lst):\n        return " ".join(lst)\n    return joinListWithSpaces(removeEmpty(splitStrOnAsterisk( addAsteriskInCaseChange(str) ))) '

In [73]:
audible_cleaned["narrator"]=audible_uncleaned["narrator"]\
                            .pipe((extract_text_left_of,"serToProcess"),strReference="Narratedby:")\
                            .pipe(np.vectorize(splitCamelCase))

In [74]:
#pd.to_timedelta(audible_uncleaned["time"])
audible_cleaned["time"]= pd.to_timedelta( 
                                        audible_uncleaned["time"]\
                                            .str.replace(" and ", " ")\
                                            .str.replace("hr", "hour")\
                                            .str.replace("mins","min")\
                                            .str.replace("Less than 1 minute","60s")\
                                        )

In [78]:
audible_cleaned["releasedate"]= pd.to_datetime(audible_uncleaned["releasedate"], format='%d-%m-%y')

In [98]:
#audible_uncleaned["stars"].str.extract(r"^(.*?)\s").pipe((pd.to_numeric,"arg"))
regex_int_or_float="\d*\.?\d*"
audible_cleaned["stars_rated"]=audible_uncleaned["stars"]\
                                                        .str.extract(r"({g1})\s".format(g1=regex_int_or_float),expand=False)\
                                                        .pipe(pd.to_numeric,errors='coerce')

In [117]:
def get_starts_rated(strSeries):
    return strSeries\
                    .str.extract(r"({g1})\s".format(g1=regex_int_or_float),expand=False)\
                    .pipe(pd.to_numeric,errors='coerce')

In [104]:
audible_cleaned["max_stars"]=audible_uncleaned["stars"]\
                                                        .str.extract(r"^(?:{g1}) out of ({g1})".format(g1=regex_int_or_float),expand=False)\
                                                        .pipe(pd.to_numeric,errors='coerce')

In [105]:
audible_cleaned["max_stars"].value_counts()

5.0    15072
Name: max_stars, dtype: int64

In [119]:
audible_cleaned["ratings"]= audible_uncleaned["stars"]\
                                                    .str.extract(r"^(?:{g1}) out of (?:{g1}) stars({g1}) rating(?:s?)".format(g1=regex_int_or_float),expand=False)\
                                                    .pipe(pd.to_numeric,errors='coerce')\
                                                    .fillna(0)

In [120]:
audible_cleaned["normalised_rating"]= audible_cleaned.eval("stars_rated / max_stars" )

In [121]:
audible_cleaned["language"]= audible_uncleaned["language"].copy()

In [125]:
audible_cleaned["price_INR"]= audible_uncleaned["price"]\
                                                        .str.replace(r",","", regex=True)\
                                                        .str.replace(r"Free","0", regex=True)\
                                                        .pipe(pd.to_numeric)

In [126]:
audible_cleaned["price_EUR_calc"] = audible_cleaned["price_INR"]*0.012 #Converted as of 5/12/2022. To do: get exchange rate from an api

In [127]:
audible_cleaned.head()

,name,author,narrator,time,releasedate,stars_rated,max_stars,ratings,normalised_rating,language,price_INR,price_EUR_cal
0,Geronimo Stilton #11 & #12,Geronimo Stilton,Bill Lobely,0 days 02:20:00,2008-08-04,5.0,5.0,34.0,1.0,English,468.0,5.616
1,The Burning Maze,Rick Riordan,Robbie Daymond,0 days 13:08:00,2018-05-01,4.5,5.0,41.0,0.9,English,820.0,9.840
2,The Deep End,Jeff Kinney,Dan Russell,0 days 02:03:00,2020-11-06,4.5,5.0,38.0,0.9,English,410.0,4.920
3,Daughter of the Deep,Rick Riordan,Soneela Nankani,0 days 11:16:00,2021-10-05,4.5,5.0,12.0,0.9,English,615.0,7.380
4,"The Lightning Thief: Percy Jackson, Book 1",Rick Riordan,Jesse Bernstein,0 days 10:00:00,2010-01-13,4.5,5.0,181.0,0.9,English,820.0,9.840


In [128]:
audible_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   name               87489 non-null  object         
 1   author             87489 non-null  object         
 2   narrator           87489 non-null  object         
 3   time               87489 non-null  timedelta64[ns]
 4   releasedate        87489 non-null  datetime64[ns] 
 5   stars_rated        15072 non-null  float64        
 6   max_stars          15072 non-null  float64        
 7   ratings            87489 non-null  float64        
 8   normalised_rating  15072 non-null  float64        
 9   language           87489 non-null  object         
 10  price_INR          87489 non-null  float64        
 11  price_EUR_cal      87489 non-null  float64        
dtypes: datetime64[ns](1), float64(6), object(4), timedelta64[ns](1)
memory usage: 8.0+ MB


In [129]:
audible_cleaned.to_csv("audible_cleaned.csv", index=False)